In [ ]:
decoder_input_ids = text_tokens.input_ids.clone()
decoder_input_ids[:, 0] = self.tokenizer.bos_token_id

所有句子（即每一行）的第一个token设置为bos_token_id，即[DEC]。

In [ ]:
labels = decoder_input_ids.masked_fill(
    decoder_input_ids == self.tokenizer.pad_token_id, -100
)

decoder_input_ids == self.tokenizer.pad_token_id，生成一个布尔张量，其形状与 decoder_input_ids 相同。
在这个布尔张量中，所有等于 pad_token_id 的位置的值为 True，其他位置的值为 False。

masked_fill 根据给定的条件将张量中的某些值替换为指定的值，在这里，pad的位置设为-100。这样做是为了在计算损失时忽略padding部分，-100是CrossEntropyLoss中的默认ignore_index。

In [ ]:
query_atts = torch.ones(query_tokens.size()[:-1], dtype=torch.long).to(image.device)

query_token可以全部注意到。

In [ ]:
attention_mask = torch.cat([query_atts, text_tokens.attention_mask], dim=1)


合并mask。

In [ ]:
lm_output = self.Qformer(
    decoder_input_ids,
    attention_mask=attention_mask,
    past_key_values=query_output.past_key_values,
    return_dict=True,
    labels=labels,
)

loss_lm = lm_output.loss

这里进行语言模型预测，计算语言模型损失。

在其内部，进行了causal mask计算以及labels移位。causal mask使得解码时只能注意到已解码的词，labels移位使得每个位置的label都是它的下一个token。详细解释见附录。